# Transformaciones 3D

In [ ]:
from ifsoac_2 import *

In [ ]:
%time Ifsoac(op={"transform":escher, "nsides":3, "p":1/2, "nrandom":10000000, "ventana_ds":1200, "cmap_ds":"bmw"}).export_images(filename_prefix="sierpEscher")

In [ ]:
@jit(nopython=True)
def totwo(p, e):
    " p.shape = (...,3); (ax,ay), (bx,by), (cx,cy) posicion en R2 de vectores unitarios en R3"
    e1, e2, e3 = e
    res=np.empty((p.shape[0], 2))
    i = 0
    for dx, dy, dz in p:
        res[i] = e1[0] * dx + e2[0] * dy + e3[0] * dz, e1[1] * dx + e2[1] * dy + e3[1] * dz
        i += 1
    return res

lz = lorenz_array3d(10000000)

In [ ]:
def plot_attractor(attractor, vent=1200, cmap=None, unit_vect = ((-10,-.5), (1,0), (0,1))):
    cols=list("xy")
    cv = ds.Canvas(plot_width = vent, plot_height = vent)
    df = pd.DataFrame( totwo(attractor, unit_vect), columns=cols)
    agg = cv.points(df, *cols)
    if cmap is None:
        cmap = palette["fire"]
    agg = tf.shade(agg, cmap=cmap)
    return tf.set_background(agg, "black")
    
plot_attractor(lz)

In [ ]:
plot_attractor(rossler_array3d(10000000), unit_vect = ((1,0), (0,1), (0,1.5)))

In [ ]:
lz_t = np.array((lz.T[0][0::3][:333333], lz.T[0][1::3][:333333], lz.T[0][2::3][:333333])).T
uv1 = ((-1,-.1), (1,-.2), (.1,1))
uv = ((1,0), (0,1), (-.5, -2.5))
plot_attractor(lz_t, unit_vect = uv)

In [ ]:
#unit_vect = {"e1":(-10,-.5), "e2":(1,0), "e3":(0,1)}
unit_vect = (-.6, -.6), (1,0), (0,1)

@jit(nopython=True)
def to_sphere(p, e):
    xyz = np.empty((p.shape[0],3))
    i = 0
    for x, y in p:
        z = 1
        r = np.sqrt(x*x+y*y+z*z)
        phi = np.arccos(z/r)
        theta = np.arctan2(y, x)
        xyz[i] = r, phi, theta
        i += 1
    return totwo(xyz, e)

@jit(nopython=True)
def from_sphere(p, e):
    "spherical coordinates (radius r, inclination θ, azimuth φ), where r ∈ [0, ∞), θ ∈ [0, π], φ ∈ [0, 2π)"
    xyz = np.empty((p.shape[0],3))
    i = 0
    minp = p.T[0].min()
    maxp = p.T[0].max()
    mint = p.T[1].min()
    maxt = p.T[1].max()
    for phi, theta in p:
        r = 1
        theta = (theta - mint) / (maxt - mint) * 2 * np.pi
        phi = (phi - minp) / (maxp - minp) * np.pi
        x = r * np.sin(phi) * np.cos(theta)
        y = r * np.sin(phi) * np.sin(theta)
        z = r * np.cos(phi)
        xyz[i] = x, y, z
        i += 1
    return totwo(xyz, e)


op = {"transform":from_sphere, "argv":unit_vect, "nsides":4, "p":1/2, 
      "nrandom":10000000, "cmap_ds":"colorwheel"}
Ifsoac(op=op).plot()

In [ ]:
op = {"transform":to_sphere, "argv":unit_vect, "nsides":4, "p":1/2, 
      "nrandom":10000000, "cmap_ds":"colorwheel"}
Ifsoac(op=op).plot()

In [ ]:
%time logi = iterar(logistica, N=10000000)
%time tienda = iterar(tent, N=10000000)
%time lorenz = lorenz_array(10000000)

In [ ]:
unit_vect =  (1,0), (0,1), (-.6, -.6)
op = {"ventana_ds":1200, "cmap_ds":"rainbow"}
%time Ifsoac([logi, tienda[::2], lorenz, lorenz[::14]], op).plot()

In [ ]:
unit_vect =  (1,0), (0,1), (-.6, -.6)
op = {"ventana_ds":1200, "transform":from_sphere, "argv":unit_vect, "cmap_ds":"rainbow"}
%time Ifsoac([logi, tienda[::2], lorenz, lorenz[::7]], op).export_images("fslorenz")

In [ ]:
@jit(nopython=True)
def from_cylinder(p, e):
    xyz = np.empty((p.shape[0], 3))
    i = 0
    minp = p.T[0].min()
    maxp = p.T[0].max()
    for phi, z in p:
        rho = 1
        phi = (phi - minp) / (maxp - minp) * 2 * np.pi
        x = rho * np.cos(phi)
        y = rho * np.sin(phi)
        z = z
        xyz[i] = x, y, z
        i += 1
    return totwo(xyz, e)


unit_vect = (1,0), (0,1), (1.6, 1.6)
op = {"ventana_ds":1200, "transform":from_cylinder, "argv":unit_vect, "cmap_ds":"rainbow"}
%time Ifsoac([logi, tienda[::2], lorenz, lorenz[::7]], op).export_images("fclogi")

In [ ]:
%time Ifsoac(op={"nsides":3, "p":1/2, "nrandom":100000000, "ventana_ds":1200, "cmap_ds":"bmw"}).export_images("Numerical_experiments_fig07")

In [ ]:
data=lorenz[::15]
cyl1 = np.vstack((np.array(Ifsoac(data, op={"x_range":(-1.5,1.5), "y_range":(-1.5,1.5)}).jDC()).T, np.linspace(0,10,len(data)))).T

   
plot_attractor(cyl1, unit_vect = ((-10,-.5), (1,0), (0,1)))

In [ ]:
from scipy.spatial.transform import Rotation as R
op = {"cmap_ds":"CET_C1", "ventana_ds":1200}
data=pd.read_csv("../../data/SUE/RL/RLMN10SUE_F4.txt", header=None).to_numpy().flatten()
cyl1 = np.vstack((np.array(Ifsoac(data, op).jDC()).T, np.linspace(0,10,len(data)))).T


li = []
for a in np.linspace(0, 2*np.pi, 120):
    r = R.from_euler('x', a)
    cyl2 = r.apply(cyl1)
    li.append(plot_attractor(cyl2, cmap=palette["CET_C1"], unit_vect = ((-5,-.5), (1,0), (0,1))))
    
for i, img in enumerate(li):
    export_image(img=img, filename=f"cylRL{i:0>3}", fmt=".png", export_path="./img/")
#tf.Images(*li)

In [ ]:
!/opt/homebrew/bin/convert -delay .1 -loop 0 img/cylRL*png "cylRL.gif"

In [ ]:
np.linspace(0, 2*np.pi, 120)[85]

In [ ]:
data=pd.read_csv("../../data/SUE/MG/MGNA5SUEF4.txt", header=None).to_numpy().flatten()
cyl1 = np.vstack((np.array(Ifsoac(data, op).jDC()).T, np.linspace(0,10,len(data)))).T


li = []
for a in np.linspace(0, 2*np.pi, 120):
    r = R.from_euler('x', a)
    cyl2 = r.apply(cyl1)
    li.append(plot_attractor(cyl2, cmap=palette["CET_C1"], unit_vect = ((-5,-.5), (1,0), (0,1))))
    
for i, img in enumerate(li):
    export_image(img=img, filename=f"cylMG{i:0>3}", fmt=".png", export_path="./img/")

In [ ]:
!/opt/homebrew/bin/convert -delay .1 -loop 0 img/cylMG*png "cylMG.gif"

In [ ]:
data=lorenz
cyl1 = np.vstack((np.array(Ifsoac(data, op).jDC()).T, np.linspace(0,10,len(data)))).T


li = []
for a in np.linspace(0, 2*np.pi, 120):
    r = R.from_euler('x', a)
    cyl2 = r.apply(cyl1)
    li.append(plot_attractor(cyl2, cmap=palette["CET_C1"], unit_vect = ((-5,-.5), (1,0), (0,1))))
    
for i, img in enumerate(li):
    export_image(img=img, filename=f"cylLZ{i:0>3}", fmt=".png", export_path="./img/")


In [ ]:
!/opt/homebrew/bin/convert -delay .1 -loop 0 cylLZ*png "cylLZ.gif"      

In [ ]:
op = {"cmap_ds":"CET_C1", "ventana_ds":1200}
data=pd.read_csv("../../data/VIGREP/RLVIGREP_511.99 Hz/RLMNVIGREP_P3.txt", header=None).to_numpy().flatten()[5000:153600]
cyl1 = np.vstack((np.array(Ifsoac(data, op).jDC()).T, np.linspace(0,10,len(data)))).T


li = []
for a in np.linspace(0, 2*np.pi, 120):
    r = R.from_euler('x', a)
    cyl2 = r.apply(cyl1)
    li.append(plot_attractor(cyl2, cmap=palette["CET_C1"], unit_vect = ((-5,-.5), (1,0), (0,1))))
    
for i, img in enumerate(li):
    export_image(img=img, filename=f"cylRL{i:0>3}", fmt=".png", export_path="./img/")
!/opt/homebrew/bin/convert -delay .1 -loop 0 img/cylRL*png "cylVRRL.gif"

In [ ]:
data=pd.read_csv("../../data/VIGREP/MGNA5VREP_511.9 Hz/MGNA5VREP_P3.txt", header=None).to_numpy().flatten()[5000:153600]
cyl1 = np.vstack((np.array(Ifsoac(data, op).jDC()).T, np.linspace(0,10,len(data)))).T


li = []
for a in np.linspace(0, 2*np.pi, 120):
    r = R.from_euler('x', a)
    cyl2 = r.apply(cyl1)
    li.append(plot_attractor(cyl2, cmap=palette["CET_C1"], unit_vect = ((-5,-.5), (1,0), (0,1))))
    
for i, img in enumerate(li):
    export_image(img=img, filename=f"cylMG{i:0>3}", fmt=".png", export_path="./img/")
!/opt/homebrew/bin/convert -delay .1 -loop 0 img/cylMG*png "cylVRMG.gif"

In [ ]:
pd.read_csv("../../data/VIGREP/RLVIGREP_511.99 Hz/RLMNVIGREP_P3.txt", header=None)[5000:153600].plot()

In [ ]:
pd.read_csv("../../data/VIGREP/MGNA5VREP_511.9 Hz/MGNA5VREP_P3.txt", header=None)[5000:153600].plot()

In [ ]:
5000/(512*60)*60

In [ ]:
200000/(512*60)

In [ ]:
512*60*5

In [ ]:
data=pd.read_csv("../../data/VIGREP/RLVIGREP_511.99 Hz/RLMNVIGREP_P3.txt", header=None).to_numpy().flatten()[5000:153600]
Ifsoac(data, op).export_images(filename_prefix="RL")

In [ ]:
data=pd.read_csv("../../data/VIGREP/MGNA5VREP_511.9 Hz/MGNA5VREP_P3.txt", header=None).to_numpy().flatten()[5000:153600]
Ifsoac(data, op).export_images(filename_prefix="MG")